In [ ]:
import os
import shutil
import re
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
import natsort


from grouping_id import find_all_identifiers_on_page
from main import main

AZURE_DI_ENDPOINT="YOUR_AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT"
AZURE_DI_KEY="YOUR_AZURE_DOCUMENT_INTELLIGENCE_KEY"
DATA_DIRECTORY = "data"


In [ ]:
UNIQUE_IDENTIFIER_KEYS = [
    "ticket",
    "ticket no",
    "notice number",
    "invoice id",
    "reference no"
]

In [ ]:
def clean_for_filename(text):
    """Removes characters that are invalid in filenames/folder names."""
    return re.sub(r'[\\/*?:"<>|]', "", text).strip()

def analyze_document(image_path, client):
    """
    Analyzes a single image using Azure Document Intelligence.
    
    Args:
        image_path (str): The full path to the image file.
        client (DocumentAnalysisClient): The Azure DI client instance.

    Returns:
        A dictionary containing extracted text, key-value pairs, tables, and the full result object,
        or None if an error occurs.
    """
    print(f"    - Analyzing: {os.path.basename(image_path)}")
    try:
        with open(image_path, "rb") as f:
            poller = client.begin_analyze_document("prebuilt-document", document=f)
            result = poller.result()
            return result
    except Exception as e:
        print(f"      [ERROR] Could not analyze {os.path.basename(image_path)}. Reason: {e}")
        return None

In [ ]:
def main(authority_name):
    """
    Main function to orchestrate the notice grouping process.
    """
    
    print("--- Starting Notice Grouping Project ---")

    # for authority_name in os.listdir(DATA_DIRECTORY):
    authority_path = os.path.join(DATA_DIRECTORY, authority_name)
    
    print(f"\nProcessing Authority: {authority_name}")
    
    image_groups = {}
    unprocessed_images = []
    
    # State variables for the current notice
    current_notice_identifier = None # Will store the tuple (key, value)
    current_group_name = None      # Will store 'notice_1', 'notice_2', etc.
    notice_counter = 0

    image_files = [f for f in os.listdir(authority_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
    sorted_images = natsorted(image_files) # Crucial for sequential processing
    
    for image_name in sorted_images:
        image_path = os.path.join(authority_path, image_name)
        
        analysis_result = analyze_document(image_path)

        # 1. Check for blank pages first
        if is_page_blank(analysis_result):
            print(f"      - Detected blank page: {image_name}. Adding to unprocessed list.")
            unprocessed_images.append(image_path)
            continue # Skip to the next image

        # 2. Find the highest-priority identifier on the page
        identifier_info = find_grouping_id(analysis_result, UNIQUE_IDENTIFIER_KEYS)

        # 3. Determine if this page starts a new notice
        if identifier_info and identifier_info != current_notice_identifier:
            # This is a new notice!
            current_notice_identifier = identifier_info
            notice_counter += 1
            current_group_name = f"notice_{notice_counter}"
            key, value = current_notice_identifier
            print(f"      - New notice detected. Assigning to folder: '{current_group_name}' (Tracking by: ('{key}', '{value}'))")

        # 4. Group the image
        if current_group_name:
            # If a notice has been started, add the current image to it.
            if current_group_name not in image_groups:
                image_groups[current_group_name] = []
            image_groups[current_group_name].append(os.path.basename(image_path))
        else:
            # This image appeared before any identifier was found.
            unprocessed_images.append(os.path.basename(image_path))

    # Now, move the grouped files into new notice folders
    if image_groups:
        # organize_files(authority_path, image_groups)
        print(image_groups)
    if unprocessed_images:
        print("\n  The following images could not be grouped (or were blank):")
        for img_path in unprocessed_images:
            print(f"    - {os.path.basename(img_path)}")